In [1]:
# Packages

import sqlite3
import mysql.connector
import pandas as pd
import numpy as np
import ast

In [3]:
# Load Data from csv

raw_recipes = pd.read_csv('data/RAW_recipes.csv')
raw_interactions = pd.read_csv('data/RAW_interactions.csv')

### Description -

#### raw_recipes.csv - 

 - name - recipe name
 - id - recipe ID
 - minutes - Minutes to prepare recipe
 - contributor_id - User ID who submitted this recipe
 - submitted - Date recipe was submitted
 - tags - Food.com tags for recipe
 - nutrition - Nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV))
 - n_steps - Number of steps in recipe
 - steps - Text for recipe steps, in order
 - description - User-provided description

#### raw_interactions.csv - 

 - user_id - User ID
 - recipe_ID - recipe ID
 - date - Date of interaction
 - rating - Rating given
 - review - Review text

In [5]:
# Arguments to pass it as a config to a connection object

config = {
  'user': 'root',
  'password': 'root@123',
  'host': 'localhost',
  'port': 3306,
  'database': 'cooking_made_easy',
  'raise_on_warnings': True
}

In [6]:
# Creating a connection to MySQL workbench

import mysql.connector

cnx = mysql.connector.connect(user='root', password='root@123',
                              host='127.0.0.1',
                              database='cooking_made_easy')

In [7]:
# Creating a cursor

cursor = cnx.cursor(dictionary=True)

In [8]:
# Cleaning data

raw_recipes.dropna(inplace=True)
raw_interactions.dropna(inplace=True)

In [9]:
# Creating dataframe with all required variables

x = raw_interactions.groupby('recipe_id')['rating'].mean()
x = pd.DataFrame(x)
x.reset_index(inplace=True)
recipes_final = raw_recipes.merge(x, how = 'inner', left_on='id', right_on='recipe_id')

In [11]:
recipes_final


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating_x,recipe_id,rating_y
0,low fat berry blue frozen dessert,38,1485,1533,09/08/99,"['weeknight', 'time-to-make', 'course', 'prepa...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand ...","this is yummy and low-fat, it always turns out...","['blueberries', 'granulated sugar', 'vanilla y...",4,4.250000,38,4.250000
1,biryani,39,265,1567,29/08/99,"['weeknight', 'time-to-make', 'course', 'main-...","[1110.7, 90.0, 81.0, 15.0, 126.0, 82.0, 28.0]",17,['soak saffron in warm milk for 5 minutes and ...,"delhi, india","['saffron', 'milk', 'hot green chili peppers',...",26,3.000000,39,3.000000
2,best lemonade,40,35,1566,05/09/99,"['60-minutes-or-less', 'time-to-make', 'course...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",8,"['into a 1 quart jar with tight fitting lid , ...",this is from one of my first good house keepi...,"['sugar', 'lemons, rind of', 'fresh water', 'f...",6,4.333333,40,4.333333
3,carina s tofu vegetable kebabs,41,1460,1586,03/09/99,"['weeknight', 'time-to-make', 'course', 'main-...","[536.1, 36.0, 128.0, 64.0, 58.0, 19.0, 21.0]",18,"['drain the tofu , carefully squeezing out exc...",this dish is best prepared a day in advance to...,"['extra firm tofu', 'eggplant', 'zucchini', 'r...",15,4.500000,41,4.500000
4,best blackbottom pie,43,140,34879,21/08/99,"['weeknight', 'time-to-make', 'course', 'cuisi...","[437.9, 29.0, 170.0, 11.0, 13.0, 54.0, 19.0]",35,"['graham cracker crust: in small bowl , combin...","sweet, chocolatey, yummy","['graham cracker crumbs', 'sugar', 'butter', '...",15,1.000000,43,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226645,bailey s chocotini,537459,10,400708,10/11/18,"['15-minutes-or-less', 'time-to-make', 'course...","[220.7, 15.0, 49.0, 2.0, 3.0, 30.0, 4.0]",5,"['to layer: add chocolate liqueur to glass', '...",a recipe that recipe complements the cocoa in ...,"['baileys irish cream', 'chocolate liqueur', '...",3,5.000000,537459,5.000000
226646,5 ingredient salted caramel crumble bars,537485,45,2000378667,12/11/18,"['60-minutes-or-less', 'time-to-make', 'course...","[52.8, 3.0, 0.0, 4.0, 1.0, 1.0, 2.0]",21,"['1', 'heat oven to 350f spray 8-inch square p...",delicious,"['pillsbury sugar cookie dough', 'caramel topp...",5,5.000000,537485,5.000000
226647,moist gingerbread cake,537543,55,2001201872,16/11/18,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",8,"['preheat the oven to 350&deg', 'f grease a lo...",a slightly sticky loaf cake flavoured with gin...,"['unsalted butter', 'applesauce', 'egg', 'unsu...",10,0.000000,537543,0.000000
226648,nutcracker peppermint red velvet cake pops,537671,135,2002198506,28/11/18,"['time-to-make', 'course', 'preparation', 'occ...","[207.9, 12.0, 93.0, 10.0, 6.0, 8.0, 10.0]",54,"['before you begin , you will need to gather t...",rich red velvet cake combines with cool pepper...,"[""devil's food cake mix"", 'eggs', 'buttermilk'...",12,0.000000,537671,0.000000


In [17]:
# Creating recipes dataframe to be exported to the recipes table in the database

recipes_df = pd.DataFrame()
recipes_df['recipe_id'] = recipes_final['id']
recipes_df['name'] = recipes_final['name']
recipes_df['description'] = recipes_final['description']
recipes_df['cooktime'] = recipes_final['minutes']
recipes_df['steps_cnt'] = recipes_final['n_steps']
recipes_df['steps'] = recipes_final['steps']
recipes_df['ingredients_cnt'] = recipes_final['n_ingredients']
recipes_df['avg_rating'] = recipes_final['rating_x']
recipes_df['submission_date'] = recipes_final['submitted']

In [18]:
recipes_df

,recipe_id,name,description,cooktime,steps_cnt,steps,ingredients_cnt,avg_rating,submission_date
0,38,low fat berry blue frozen dessert,"this is yummy and low-fat, it always turns out...",1485,13,"['toss 2 cups berries with sugar', 'let stand ...",4,4.250000,09/08/99
1,39,biryani,"delhi, india",265,17,['soak saffron in warm milk for 5 minutes and ...,26,3.000000,29/08/99
2,40,best lemonade,this is from one of my first good house keepi...,35,8,"['into a 1 quart jar with tight fitting lid , ...",6,4.333333,05/09/99
3,41,carina s tofu vegetable kebabs,this dish is best prepared a day in advance to...,1460,18,"['drain the tofu , carefully squeezing out exc...",15,4.500000,03/09/99
4,43,best blackbottom pie,"sweet, chocolatey, yummy",140,35,"['graham cracker crust: in small bowl , combin...",15,1.000000,21/08/99
...,...,...,...,...,...,...,...,...,...
226645,537459,bailey s chocotini,a recipe that recipe complements the cocoa in ...,10,5,"['to layer: add chocolate liqueur to glass', '...",3,5.000000,10/11/18
226646,537485,5 ingredient salted caramel crumble bars,delicious,45,21,"['1', 'heat oven to 350f spray 8-inch square p...",5,5.000000,12/11/18
226647,537543,moist gingerbread cake,a slightly sticky loaf cake flavoured with gin...,55,8,"['preheat the oven to 350&deg', 'f grease a lo...",10,0.000000,16/11/18
226648,537671,nutcracker peppermint red velvet cake pops,rich red velvet cake combines with cool pepper...,135,54,"['before you begin , you will need to gather t...",12,0.000000,28/11/18


In [20]:
from pathlib import Path
filepath = Path(
    '/Users/saurabh/workspace/Spring_2022/ADT/Cooking-made-easy/data/recipes_new.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
recipes_df.to_csv(filepath)


In [21]:
# Creating function to get unique list of values

def calc_unique(column):
    unique_item = set()
    for row in column:
        row = ast.literal_eval(row)
        for i in row:
            unique_item.add(i)
            
    return unique_item

In [22]:
# Creating tags dataframe

unique_tags = calc_unique(recipes_final['tags'])

main_tag = []
for tag in unique_tags:
    if ('less' in tag or 'more' in tag) and '-' in tag:
        main_tag.append(tag)
        
other_tags = ['potluck','weeknight','dinner-party','appetizers', 'for-1-or-2',
 'for-large-groups', 'time-to-make', 'to-go', 'toddler-friendly', 'for-large-groups-holiday-event']
main_tag.extend(other_tags)

main_tags = {'tag_id': list(range(1,len(main_tag)+1)),'name': main_tag}
tags_df = pd.DataFrame(main_tags)

In [31]:
filepath = Path(
    '/Users/saurabh/workspace/Spring_2022/ADT/Cooking-made-easy/data/tags.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
tags_df.to_csv(filepath)


In [36]:
# Creating tag_to_recipe dataframe

tag_recipe = recipes_final[['id', 'tags']]

tags_arr = []
tags_id = []

for index, row in tag_recipe.iterrows():
    row['tags'] = row['tags'].replace("[","")
    row['tags'] = row['tags'].replace("]","")
    row['tags'] = row['tags'].replace("'","")
    row['tags'] = row['tags'].replace(" ","")
    
    tag = [j for j in row['tags'].split(',') if j in list(tags_df['name'])]

    tags_arr.append(tag)
    tags_id.append(row['id'])

tag_fin_recipe = pd.DataFrame({'tag_id': tags_arr, 'recipe_id': tags_id})
tag_fin_recipe = tag_fin_recipe.explode('tag_id')
tag_to_recipe = tag_fin_recipe.merge(tags_df, how = 'inner', left_on='tag_id', right_on='name')

In [38]:
filepath = Path(
    '/Users/saurabh/workspace/Spring_2022/ADT/Cooking-made-easy/data/tag_to_recipe.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
tag_to_recipe.to_csv(filepath)


In [32]:
# Creating ingredients dataframe

unique_ingredients = list(calc_unique(recipes_final['ingredients']))

ingredients = {'id_ingredients': list(range(1,len(unique_ingredients)+1)),'name': unique_ingredients}
ingredients_df = pd.DataFrame(ingredients)

In [34]:
filepath = Path(
    '/Users/saurabh/workspace/Spring_2022/ADT/Cooking-made-easy/data/ingredients.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
ingredients_df.to_csv(filepath)


In [79]:
df = pd.read_csv('data/ind.csv')


In [80]:
df['ingredients'] = df['ingredients'].astype('string')
df['ingredients1'] = df['ingredients'].apply(ast.literal_eval)
df
# ind_data


,id,ingredients,ingredients1
0,38,"['blueberries', 'granulated sugar', 'vanilla y...","[blueberries, granulated sugar, vanilla yogurt..."
1,39,"['saffron', 'milk', 'hot green chili peppers',...","[saffron, milk, hot green chili peppers, onion..."
2,40,"['sugar', 'lemons, rind of', 'fresh water', 'f...","[sugar, lemons, rind of, fresh water, fresh le..."
3,41,"['extra firm tofu', 'eggplant', 'zucchini', 'r...","[extra firm tofu, eggplant, zucchini, red pepp..."
4,43,"['graham cracker crumbs', 'sugar', 'butter', '...","[graham cracker crumbs, sugar, butter, cornsta..."
...,...,...,...
231632,537459,"['baileys irish cream', 'chocolate liqueur', '...","[baileys irish cream, chocolate liqueur, vodka]"
231633,537485,"['pillsbury sugar cookie dough', 'caramel topp...","[pillsbury sugar cookie dough, caramel topping..."
231634,537543,"['unsalted butter', 'applesauce', 'egg', 'unsu...","[unsalted butter, applesauce, egg, unsulphured..."
231635,537671,"[""devil's food cake mix"", 'eggs', 'buttermilk'...","[devil's food cake mix, eggs, buttermilk, red ..."


In [81]:
ind_data = df.explode('ingredients1')

In [82]:
del ind_data['ingredients']
ind_data


,id,ingredients1
0,38,blueberries
0,38,granulated sugar
0,38,vanilla yogurt
0,38,lemon juice
1,39,saffron
...,...,...
231636,537716,blue cheese
231636,537716,cheez whiz
231636,537716,rolls
231636,537716,green onion


In [77]:
ind_data_1 = ind_data.iloc[:, 1:]
ind_data


,ingredients_name
0,blueberries
0,granulated sugar
0,vanilla yogurt
0,lemon juice
1,saffron
...,...
231636,blue cheese
231636,cheez whiz
231636,rolls
231636,green onion


In [83]:
ind_data = ind_data.rename(columns={'id': 'recipe_id', 'ingredients1': 'ingredients_name'})
ind_data


,recipe_id,ingredients_name
0,38,blueberries
0,38,granulated sugar
0,38,vanilla yogurt
0,38,lemon juice
1,39,saffron
...,...,...
231636,537716,blue cheese
231636,537716,cheez whiz
231636,537716,rolls
231636,537716,green onion


In [84]:
ind_data = ind_data[['ingredients_name', 'recipe_id']]
ind_data

,ingredients_name,recipe_id
0,blueberries,38
0,granulated sugar,38
0,vanilla yogurt,38
0,lemon juice,38
1,saffron,39
...,...,...
231636,blue cheese,537716
231636,cheez whiz,537716
231636,rolls,537716
231636,green onion,537716


In [85]:
filepath = Path(
    '/Users/saurabh/workspace/Spring_2022/ADT/Cooking-made-easy/data/ingredients_map.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
ind_data.to_csv(filepath, index=False)


In [59]:

def calc_unique(df):
    df['ingredients'] = df['ingredients'].astype('string')
    df['ingredients1'] = df['ingredients'].apply(ast.literal_eval)
    return df


In [ ]:
# Closing the connection

cnx.close()